<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/few_shot_bert_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
!pip install deeppavlov

In [94]:
!pip install tensorflow==1.15.0

In [95]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [96]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-8awr1kjb
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-8awr1kjb
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=88468b19c758a754fc409a63353a531c37bfe17e5846f5cb529c7796077bf7f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-3qz91yqx/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [97]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [98]:
import csv
import pandas as pd
import re
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [99]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-22 22:00:33--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-22 22:00:33 ERROR 404: Not Found.

--2021-05-22 22:00:34--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1192227 (1.1M) [text/plain]
Saving to: ‘new_anns60k.csv.15’

new_anns60k.csv.15  100%[===================>]   1.14M  --.-KB/s    in 0.02s   

2

In [100]:
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

In [101]:
with open('new_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
      if len(new_row) > 9:
        new_row = new_row[:9]
      writer.writerow(new_row)

In [102]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0


In [103]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

In [104]:
df = cor_df.iloc[:9500,:]

In [105]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']

In [106]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",0,0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,0
8450,сероквель,"[0.087633356, 0.93445277, 0.69432694, 0.298702...",0,6,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[-0.04207938, 0.36281538, 0.25962716, 0.010672...","[0.087633356, 0.93445277, 0.69432694, 0.298702...",0,0
8541,день,"[0.11187396, 1.2210127, -0.88301444, -0.466371...",0,7,0,0,2,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,"[0.11187396, 1.2210127, -0.88301444, -0.466371...","[0.10426845, 0.58591, -0.40001512, -0.31153095...","[0.3036255, 0.7870125, -0.33815098, -0.5249253...",0,0


In [107]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [108]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col

In [109]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [110]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [111]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'нексиум', 'париет', 'желудке', 'пробиотики', 'схемы', 'обсудить',
        'химиком', 'терапевтом', 'паллиативным', 'понадобятся'],
       dtype='object', length=1795),
 'HEAD_ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'HEAD_

In [112]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [113]:
df = df[['TOKEN', 'NER']].iloc[:5000,:]

In [114]:
lst = []
for i in df.NER:
  if i == 1.0:
    lst.append('B-MED')
  elif i == 2.0:
    lst.append('B-NOMIN')
  elif i == 3.0:
    lst.append('B-PRON')
  else:
    lst.append('0')
df.NER = lst

In [115]:
train, test \
    = train_test_split(df, random_state=1)

train, val \
    = train_test_split(train,  random_state=1) # 0.25 x 0.8 = 0.2

In [116]:
train

,TOKEN,NER
508,страх,0
3048,тысяче,0
2781,аппетит,0
4840,19:23,0
4586,дозой,0
...,...,...
3056,заслуги,0
4566,со,0
777,что,0
4682,рекомендуют,0


In [117]:
test 

,TOKEN,NER
2764,кг,0
4767,же,0
3814,szt,0
3499,при,0
2735,явления,0
...,...,...
3015,корпорацию,0
1891,кто,0
2773,"0,5",0
3803,mg,0


In [118]:
val

,TOKEN,NER
2723,ли,0
1029,ли,0
4435,потом,0
908,на,0
3330,врачу,0
...,...,...
1980,сильного,0
537,после,0
2246,подавших,0
2806,может,0


In [119]:
train.to_csv('tmp_train.txt', sep=' ', header=False, index=False)
test.to_csv('tmp_test.txt', sep=' ', header=False, index=False)
val.to_csv('tmp_valid.txt', sep=' ', header=False, index=False)

In [120]:
# train.to_csv('train.txt', sep=' ', header=False, index=False)
# test.to_csv('test.txt', sep=' ', header=False, index=False)
# val.to_csv('valid.txt', sep=' ', header=False, index=False)

In [121]:
# print(open('train.txt').readlines()[0].split())
# open('valid.txt').readlines()[10].split()

In [122]:
# handler = open('tmp_train.txt')
# wr = open('train.txt', 'w')
# # print(len(handler.readlines()))
# for i in handler.readlines():
#   i = i.replace('\t', ' ')
#   i = i.replace('O', '0')
#   wr.write(i)
# wr.close()
# handler.close

In [123]:
import re

In [124]:
len(open('tmp_test.txt', encoding='utf-8').readlines())

1250

In [125]:
handler = open('tmp_test.txt', encoding='utf-8')
wr = open('test.txt', 'w', encoding='utf-8')
ll = ['B-MED', 'I-MED', 'B-PRON', 'I-EFFECT', 'B-NOMIN', 'I-SICK', '0', 0]
# print(len(handler.readlines()))
for i in handler.readlines():
  i = i.replace('\t', ' ')
  if i == '\n':
    # print('yep')
    wr.write('\n')
  if i.split() and len(i.split()) > 1:
    if re.match(r'[A-zА-я]+', i.split()[0]) and i.split()[1] in ll:
      wr.write(f'{i.split()[0]} {i.split()[1]}\n')
wr.close()
handler.close

<function TextIOWrapper.close>

In [126]:
len(open('tmp_test.txt', encoding='utf-8').readlines())

1250

In [127]:
handler = open('tmp_train.txt', encoding='utf-8')
wr = open('train.txt', 'w', encoding='utf-8')
ll = ['B-MED', 'I-MED', 'B-PRON', 'I-EFFECT', 'B-NOMIN', 'I-SICK', '0', 0]
# print(len(handler.readlines()))
for i in handler.readlines():
  i = i.replace('\t', ' ')
  if i == '\n':
    # print('yep')
    wr.write('\n')
  if i.split() and len(i.split()) > 1:
    if re.match(r'[A-zА-я]+', i.split()[0]) and i.split()[1] in ll:
      wr.write(f'{i.split()[0]} {i.split()[1]}\n')
wr.close()
handler.close

<function TextIOWrapper.close>

In [128]:
handler = open('tmp_valid.txt', encoding='utf-8')
wr = open('valid.txt', 'w', encoding='utf-8')
ll = ['B-MED', 'I-MED', 'B-PRON', 'I-EFFECT', 'B-NOMIN', 'I-SICK', '0', 0]
# print(len(handler.readlines()))
for i in handler.readlines():
  i = i.replace('\t', ' ')
  if i == '\n':
    # print('yep')
    wr.write('\n')
  if i.split() and len(i.split()) > 1:
    if re.match(r'[A-zА-я]+', i.split()[0]) and i.split()[1] in ll:
      wr.write(f'{i.split()[0]} {i.split()[1]}\n')
wr.close()
handler.close

<function TextIOWrapper.close>

In [129]:
print(open('train.txt').readlines()[0].split())
print(open('test.txt').readlines()[1].split())
print(open('valid.txt').readlines()[1].split())

['страх', '0']
['же', '0']
['ли', '0']


In [130]:
print(len(open('train.txt').readlines()))

2664


In [131]:
# !rm -r logs  tag.dict	test.txt  valid.txt
# !ls

In [132]:
# !wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/test.txt
# !wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/valid.txt

In [133]:
# !cp train.txt test.txt
# !cp train.txt valid.txt
#  !rm -r logs  tag.dict	test.txt  train.txt  valid.txt
# !rm test.txt  train.txt  valid.txt
# !rm tmp_test.txt	tmp_train.txt tmp_valid.txt

!ls

content		    new_anns60k.csv.13	new_anns60k.csv.6   test.txt
logs		    new_anns60k.csv.14	new_anns60k.csv.7   tmp_test.txt
new_anns60k.csv     new_anns60k.csv.15	new_anns60k.csv.8   tmp_train.txt
new_anns60k.csv.1   new_anns60k.csv.2	new_anns60k.csv.9   tmp_valid.txt
new_anns60k.csv.10  new_anns60k.csv.3	new_df_anns60k.csv  train.txt
new_anns60k.csv.11  new_anns60k.csv.4	sample_data	    valid.txt
new_anns60k.csv.12  new_anns60k.csv.5	tag.dict	    valid.txt.1


In [134]:
import os
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['train']['epochs'] = 3 # !!!тут обычно 30, но после того, как все грохнулось, временно будет 10
os.environ["CUDA_VISIBLE_DEVICES"]='3'
ner_config['train']['pytest_batch_size']= 10
ner_config['device']='GPU'
ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]
ner_config['train']['batch_size'] = 10

In [135]:
# ner_config

In [136]:
ner_config['train']['epochs']

3

In [137]:
# ner_config
open('train.txt').readlines()[-1], open('test.txt').readlines()[-1], open('valid.txt').readlines()[-1]

('под 0\n', 'на 0\n', 'в 0\n')

In [ ]:
ner_model = train_model(ner_config, download=True)

2021-05-22 22:01:52.47 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2021-05-22 22:01:52.63 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2021-05-22 22:01:52.508 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/tag.dict]
2021-05-22 22:01:52.521 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/tag.dict]


In [ ]:
ner_model.save()

In [ ]:
ner_model(["какое-то не лекарство", "пью леонезит 1 месяц", "варкрафт 2 интересная игра"])

In [ ]:
nanika = ner_model(['Паксил норм. Принимал около года, щас перерыв. \
  Синдром отмены не то чтобы совсем отсутствует, но мне особых непрятностей не доставил. \
  Еще тироксин в качестве антидепрессанта хорош, хотя формально показания у него другие.'])

In [ ]:
list(zip(nanika[0][0], nanika[1][0]))

In [ ]:
# !rm checkpoint -r model.data-00000-of-00001  model.meta logs	tag.dict     model.index		       sample_data

In [ ]:
from google.colab import files
import os
!ls

In [ ]:
for i in os.listdir():
  print(i)
  files.download(i)